In [1]:
import os
from py2neo import Graph, Node, Relationship
from tqdm import tqdm



In [2]:

user = os.getenv("DB_USER")
password = os.getenv("DB_PASS")

graph_db: Graph = Graph("bolt://localhost:7687", auth=(user, password))

### lista para o algoritmos ###
* passar uma lista de pontos de interesse junto com o ponto inicial 
* calcular a distancia relativa entre os pontos
* testar todas as possibilidades de caminhos ate encontrar o menor (força bruta)

In [ ]:
#achar a rua mais perto do ponto passado
def findRoadId(latitude: float, longitude: float):
   return graph_db.run(
    f'''
    WITH {latitude} AS targetLat, {longitude} AS targetLon  

    MATCH (a)-[r:ROAD_TO]->(b)
    WHERE a.latitude IS NOT NULL AND a.logintude IS NOT NULL
    AND b.latitude IS NOT NULL AND b.logintude IS NOT NULL

    WITH r, a, b,
        point({{y: a.latitude, x: a.logintude}}) AS pointA,
        point({{y: b.latitude, x: b.logintude}}) AS pointB,
        point({{y: targetLat, x: targetLon}}) AS targetPoint

    WITH r, a, b,
        point.distance(pointA, targetPoint) AS distA,
        point.distance(pointB, targetPoint) AS distB

    WITH r, a, b,
        CASE WHEN distA < distB THEN distA ELSE distB END AS minDist

    ORDER BY minDist ASC
    LIMIT 1

    RETURN id(r)
    '''
)


In [ ]:
#achar o ponto mais perto entre os pontos de interesse

In [15]:
initial_point = [-23.2848682,-47.6720885]

[lat, long] = initial_point

initialRoad = findRoadId(lat, long)

target_points = [
    findRoadId(latitude=-23.2999866, longitude=-47.6650897),
    findRoadId(latitude=-23.2829741, longitude=-47.6745936),
    findRoadId(latitude=-23.2847718, longitude=-47.6730844)
]

display(initialRoad)

id(r)
806
